In [ ]:
/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_186w_clean_8b_sv.jsonl

In [24]:
#分类通用
import json

from tqdm import tqdm
def evaluation_metrics(data_path):
        
    base = [json.loads(q) for q in open(data_path, "r")]
    correct=0
    incorrect=0
    #print(base)
    for answers in tqdm(base):
        gt=answers['ground_truth'].split('.')[0].replace(' ','').lower()
        answer=answers['text'].split('.')[0].replace(' ','').lower()
        if gt==answer:
                correct=correct+1
        else:
                incorrect=incorrect+1
        # else:
        #     continue
    print('correct:',correct)
    print('incorrect:',incorrect)
    print('Total:',correct+incorrect)
    print('Acc:',(correct/(correct+incorrect)))
    
evaluation_metrics("/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_186w_clean_8b_sv.jsonl")

evaluation_metrics("/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_sharegpt4v_186w_clean_8B_llava_instruct150k.jsonl")

evaluation_metrics("/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_186w_clean_8B_llava_instruct150k.jsonl")

  0%|          | 0/6500 [00:00<?, ?it/s]

100%|██████████| 6500/6500 [00:00<00:00, 558690.44it/s]


correct: 5876
incorrect: 624
Total: 6500
Acc: 0.904


100%|██████████| 6500/6500 [00:00<00:00, 931971.97it/s]


correct: 4642
incorrect: 1858
Total: 6500
Acc: 0.7141538461538461


100%|██████████| 2972/2972 [00:00<00:00, 1248794.98it/s]

correct: 411
incorrect: 2561
Total: 2972
Acc: 0.1382907133243607


In [1]:
import json
import pandas as pd
import   re
json_file_path = '/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_186w_clean_8b_sv.jsonl'

# 读取JSON文件\
data = []
with open(json_file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))
        


# 遍历JSON数据，提取并转换每个元素的text字段为元组
coordinates_list = []  # 用于存储转换后的元组列表

pred_coordinates_list = [] 
true_coordinates_list = [] 

for item in data:
    try:
        coordinates_str = item['text'].split('(')[1].split(')')[0]  # 提取括号内的坐标字符串
        true_coordinates_str = item['ground_truth'].split('(')[1].split(')')[0]  # 提取括号内的坐标字符串

        pred_coordinates_list.append((float(coordinates_str.split(',')[0]),float(coordinates_str.split(',')[1])) ) # 将元组添加到列表中
        true_coordinates_list.append((float(true_coordinates_str.split(',')[0]),float(true_coordinates_str.split(',')[1])))
    except:
        continue
# Compute accuracy according to PlaNet criteria
# Street = 1km
# City = 25km
# Region = 200km
# Country = 750km
# Continent = 2500km
import math
#import webdataset as wds
import s2sphere
import tqdm


#another way to compute the distance
import numpy as np
def getDist_points(q, v):
    EARTH_REDIUS = 6378.137
    v=np.array(v).reshape(-1,2)
    q=np.array(q).reshape(-1,2)
    v, q = np.deg2rad(v), np.deg2rad(q)
    
    lat1, lat2 = v[:,1].reshape(-1,1), q[:,1].reshape(-1,1)
    lng1, lng2 = v[:,0].reshape(-1,1), q[:,0].reshape(-1,1)
    sin_lats = np.dot(np.sin(lat1), np.sin(lat2.T))
    cos_lats = np.dot(np.cos(lat1), np.cos(lat2.T))
    cos_lng_diff = np.cos(lng2.reshape(-1) - lng1.reshape(-1,1))
    s = np.arccos(sin_lats + cos_lats*cos_lng_diff)
    d = s * EARTH_REDIUS ###* 1000
    d = d.T # q to v
    return d

#like above way to compute the distance
def get_two_point_distance(lat1, lon1, lat2, lon2):
    EARTH_REDIUS = 6378.137
    
    #deg2rad
    
    sin_lats = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2))
    cos_lats = math.cos(math.radians(lat1)) * math.cos(math.radians(lat2))
    cos_lng_diff = math.cos(math.radians(lon2) - math.radians(lon1))
    s = math.acos(sin_lats + cos_lats * cos_lng_diff)
    d = s * EARTH_REDIUS
    return d



def compute_accuracy_planet(pred_coordinates_list, true_coordinates_list):
    accuracy_planet = {
        "street": 0,
        "city": 0,
        "region": 0,
        "country": 0,
        "continent": 0,
    }

    for idx in range(len(pred_coordinates_list)):
        

        #distance_km = get_two_point_distance(pred_coordinates_list[idx][0], pred_coordinates_list[idx][1], true_coordinates_list[idx][0], true_coordinates_list[idx][1])
        #distance_km = getDist_points([pred_coordinates_list[idx][0], pred_coordinates_list[idx][1]], [true_coordinates_list[idx][0], true_coordinates_list[idx][1]])
        
        distance_km = getDist_points([pred_coordinates_list[idx][1], pred_coordinates_list[idx][0]], [true_coordinates_list[idx][1], true_coordinates_list[idx][0]])
        distance_km=distance_km[0][0]
        if distance_km <= 1:
            accuracy_planet["street"] += 1
        if distance_km <= 25:
            accuracy_planet["city"] += 1
        if distance_km <= 200:
            accuracy_planet["region"] += 1
        if distance_km <= 750:
            accuracy_planet["country"] += 1
        if distance_km <= 2500:
            accuracy_planet["continent"] += 1

    for k in accuracy_planet.keys():
        accuracy_planet[k] /= len(true_coordinates_list)
        accuracy_planet[k] *= 100.0 # percent

    return accuracy_planet

print(compute_accuracy_planet(pred_coordinates_list, true_coordinates_list))

print(len(pred_coordinates_list))




{'street': 1.6153846153846154, 'city': 83.96923076923078, 'region': 90.43076923076923, 'country': 93.23076923076923, 'continent': 94.18461538461538}
6500


In [3]:
pred_coordinates_list[0][1], pred_coordinates_list[0][0]

(18.540673, -33.947932)

In [31]:
print(len(true_coordinates_list))
print(len(pred_coordinates_list))
print(len(data))

6466
6466
6500


In [45]:
json_file_path = '/data3/zhangxin/wuwen/vila-1.5/citybench/sv/vila_train_all_sisv_pretrain_att_186w_clean_8b_sv.jsonl'

# 读取JSON文件\
data = []
with open(json_file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))
        


# 遍历JSON数据，提取并转换每个元素的text字段为元组
coordinates_list = []  # 用于存储转换后的元组列表

pred_coordinates_list = [] 
true_coordinates_list = [] 

for item in data:
    
    
    coordinates_str = item['text'].split('(')[1].split(')')[0]  # 提取括号内的坐标字符串
    true_coordinates_str = item['ground_truth'].split('(')[1].split(')')[0]  # 提取括号内的坐标字符串
    lat,lon=re.findall(r"[-+]?\d*\.\d+|\d+", coordinates_str)
    pred_coordinates_list.append((float(coordinates_str.split(',')[0]),float(coordinates_str.split(',')[1])) ) # 将元组添加到列表中
    true_coordinates_list.append((float(true_coordinates_str.split(',')[0]),float(true_coordinates_str.split(',')[1])))


In [46]:
coordinates_str

'40.710673, -73.972937'

In [35]:
lon

'-46.593729'

In [34]:
lat

'-23.593651'